<hr style="border:2px solid #808080"> </hr>
<center><h1 style="color:#03122E;"> Álgebra Lineal Numérica IMT2111</h1></center>
<center><h1 style="color:#173F8A;"> Capítulo 2</h3></center>
<center><h1 style="color:#0176DE;"> Prof. Manuel A. Sánchez</h3></center>
<hr style="border:2px solid #808080"> </hr>

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.la import EigenValues_Preconditioner
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Solution
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))
fes = H1(mesh, order=1, dirichlet='left|bottom')
u, v = fes.TnT()
a = BilinearForm(InnerProduct(grad(u), grad(v))*dx).Assemble()
b = LinearForm(1*v*dx).Assemble()
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs())*b.vec
# Draw(gfu)

### Set Poisson problem 

In [ ]:
def SetPoisson(mesh, f=1, p=1, dirichlet_bndry=None, condense=False):
    # H1-conforming Finite Element Space
    fes = H1(mesh, order=p, dirichlet=dirichlet_bndry)
    # Trial and Test Functions
    u, v = fes.TnT()
    # Bilinear form 
    a = BilinearForm(fes, symmetric=True, condense=condense)
    a += (grad(u)*grad(v))*dx 
    # Linear form
    b = LinearForm(f*v*dx)
    # Gridfunction
    gfu = GridFunction(fes)
    return fes, a, b, gfu

### Direct solver

In [4]:
def SolvePoissonDirect(a, b, gfu, fes, condense=False):
    a.Assemble()
    b.Assemble()
    # Direct solver
    inv = a.mat.Inverse(freedofs=fes.FreeDofs(coupling=condense))
    # Solve steps depend on condense
    if condense:
        b.vec.data += a.harmonic_extension_trans * b.vec
        gfu.vec.data = inv * b.vec
        gfu.vec.data += a.harmonic_extension * gfu.vec
        gfu.vec.data += a.inner_solve * b.vec
    else: 
        gfu.vec.data = inv * b.vec
    return gfu, fes.ndof

### Iterative method : Conjugate Gradient solver (no preconditioner)

In [5]:
def SolvePoissonIterativeCG(a, b, gfu, fes, condense=False, compute_condnum=False):
    a.Assemble()
    b.Assemble()
#     inv = CGSolver(a.mat, pre=NoPrec(a,fes), maxsteps=10000)
    preI = Projector(mask=fes.FreeDofs(coupling=condense), range=True)
    inv = CGSolver(a.mat, pre=preI, maxsteps=10000)
    
    # Solve steps depend on condense
    if condense:
        b.vec.data += a.harmonic_extension_trans * b.vec
        gfu.vec.data = inv * b.vec
        gfu.vec.data += a.harmonic_extension * gfu.vec
        gfu.vec.data += a.inner_solve * b.vec
    else: 
        gfu.vec.data = inv * b.vec
    if compute_condnum is False:
        return gfu, inv.GetSteps()
    else:
        lams = EigenValues_Preconditioner(mat=a.mat, pre=preI)
        kappa = max(lams)/min(lams)
        return gfu, ((inv.GetSteps(), kappa))

In [ ]:
# class Preconditioner identity
class NoPrec(BaseMatrix):
    def __init__ (self, a, fes):
        super(NoPrec, self).__init__()
        self.a = a
        self.fes = fes
    def Mult (self, x, y):
        y[:] = 0.0
        FreeDofsindex = np.where(self.fes.FreeDofs())[0]
        y.FV().NumPy()[FreeDofsindex] = x.FV().NumPy()[FreeDofsindex]
    def Height (self):
        return self.a.mat.shape[0]
    def Width (self):
        return self.a.mat.shape[1]

In [ ]:
class Jacobi(BaseMatrix):
    def __init__ (self, smoother):
        super(Jacobi, self).__init__()
        self.smoother = smoother
    def Mult (self, x, y):
        y[:] = 0.0
        self.smoother.Mult(x, y)
    def Height (self):
        return self.smoother.height
    def Width (self):
        return self.smoother.height


In [ ]:
class GaussSeidelL(BaseMatrix):
    def __init__ (self, smoother):
        super(GaussSeidelL, self).__init__()
        self.smoother = smoother
    def Mult (self, x, y):
        y[:] = 0.0
        self.smoother.Smooth(y, x)
    def Height (self):
        return self.smoother.height
    def Width (self):
        return self.smoother.height

In [ ]:
class GaussSeidelU(BaseMatrix):
    def __init__ (self, smoother, steps=1):
        super(GaussSeidelU, self).__init__()
        self.smoother = smoother
        self.steps = steps
    def Mult (self, x, y):
        y[:] = 0.0
        self.smoother.SmoothBack(y, x)
    def Height (self):
        return self.smoother.height
    def Width (self):
        return self.smoother.height

In [ ]:
class GaussSeidelSym(BaseMatrix):
    def __init__ (self, smoother, steps=1):
        super(GaussSeidelSym, self).__init__()
        self.smoother = smoother
        self.steps = steps
    def Mult (self, x, y):
        y[:] = 0.0
        self.smoother.Smooth(y, x)
        self.smoother.SmoothBack(y, x)
    def Height (self):
        return self.smoother.height
    def Width (self):
        return self.smoother.height

In [ ]:
class ILU(BaseMatrix):
    # Give a scipy.sparse mat
    def __init__ (self, a):
        super(ILU, self).__init__()
        rows,cols,vals = a.mat.COO()
        A = sp.csr_matrix((vals,(rows,cols))) # NGSolve mat to scipy.sparse
        self.A = A
        self.ilu = spilu(A)
    def Mult (self, x, y):
        x_vec = x.FV().NumPy()
        y_vec = self.ilu.solve(x_vec)
        y.FV()[:] = y_vec
    def Height (self):
        return self.A.shape[0]
    def Width (self):
        return self.A.shape[1]